# SPECFEM2D Parameter File (DATA/Par_file)

## Simulation Type

 - Simulations parameters, `SIMULATION_TYPE` is `1` for forward runs and `3` for kernel computation for using adjoint simulations and `SAVE_FORWARD` is used in preparation for adjoint simulations.
 - `NPROC` is used for parallelization using `mpi`.
 
 ```toml
SIMULATION_TYPE                 = 1
# 0 = regular wave propagation simulation, 1/2/3 = noise simulation
NOISE_TOMOGRAPHY                = 0
# save the last frame, needed for adjoint simulation
SAVE_FORWARD                    = .false.

# parameters concerning partitioning
NPROC                           = 1              # number of processes
```

## Time Step
 - `DT` is time step. `NSTEP` is total number of time step. Resulting records will be `DT*NSTEP` long.
 - [How to choose the time step](https://github.com/geodynamics/specfem2d/wiki/04_running_the_solver#how-to-choose-the-time-step=)

```toml
# time step parameters
# total number of time steps
NSTEP                           = 1600

# duration of a time step (see section "How to choose the time step" of the manual for how to do this)
DT                              = 1.1d-3

# time stepping
# 1 = Newmark (2nd order), 2 = LDDRK4-6 (4th-order 6-stage low storage Runge-Kutta), 3 = classical RK4 4th-order 4-stage Runge-Kutta
time_stepping_scheme            = 1
```

## Calculation type
- Simulation are performed in `xz`-plane can work with P-SV or SH waves.
 - There is also support axisymmetric simulations.

```toml
# set the type of calculation (P-SV or SH/membrane waves)
P_SV                            = .true.

# axisymmetric (2.5D) or Cartesian planar (2D) simulation
AXISYM                          = .false.
```

## Mesh
```toml
# Partitioning algorithm for decompose_mesh
PARTITIONING_TYPE               = 3              # SCOTCH = 3, ascending order (very bad idea) = 1

# number of control nodes per element (4 or 9)
NGNOD                           = 9

# creates/reads a binary database that allows to skip all time consuming setup steps in initialization
# 0 = does not read/create database
# 1 = creates database
# 2 = reads database
setup_with_binary_database      = 0
```

## Model


```toml
# available models
#   default       - define model using nbmodels below
#   ascii         - read model from ascii database file
#   binary        - read model from binary databse file
#   binary_voigt  - read Voigt model from binary database file
#   external      - define model using define_external_model subroutine
#   gll           - read GLL model from binary database file
#   legacy        - read model from model_velocity.dat_input
MODEL                           = default

# Output the model with the requested type, does not save if turn to default or .false.
# (available output formats: ascii,binary,gll,legacy)
SAVE_MODEL                      = default
```

## Attenuation

```toml
# attenuation parameters
ATTENUATION_VISCOELASTIC        = .false.        # turn attenuation (viscoelasticity) on or off for non-poroelastic solid parts of the model
ATTENUATION_VISCOACOUSTIC       = .false.        # turn attenuation (viscoacousticity) on or off for non-poroelastic fluid parts of the model

# for viscoelastic or viscoacoustic attenuation
N_SLS                           = 3              # number of standard linear solids for attenuation (3 is usually the minimum)
ATTENUATION_f0_REFERENCE        = 5.196          # in case of attenuation, reference frequency in Hz at which the velocity values in the velocity model are given (unused otherwise); relevant only if source is a Dirac or a Heaviside, otherwise it is automatically set to f0 the dominant frequency of the source in the DATA/SOURCE file
READ_VELOCITIES_AT_f0           = .false.        # read seismic velocities at ATTENUATION_f0_REFERENCE instead of at infinite frequency (see user manual for more information)
USE_SOLVOPT                     = .false.        # use more precise but much more expensive way of determining the Q factor relaxation times, as in https://doi.org/10.1093/gji/ggw024

# for poroelastic attenuation
ATTENUATION_PORO_FLUID_PART     = .false.        # turn viscous attenuation on or off for the fluid part of poroelastic parts of the model
Q0_poroelastic                  = 1              # quality factor for viscous attenuation (ignore it if you are not using a poroelastic material)
freq0_poroelastic               = 10             # frequency for viscous attenuation (ignore it if you are not using a poroelastic material)

# to undo attenuation and/or PMLs for sensitivity kernel calculations or forward runs with SAVE_FORWARD
# use the flag below. It performs undoing of attenuation and/or of PMLs in an exact way for sensitivity kernel calculations
# but requires disk space for temporary storage, and uses a significant amount of memory used as buffers for temporary storage.
# When that option is on the second parameter indicates how often the code dumps restart files to disk (if in doubt, use something between 100 and 1000).
UNDO_ATTENUATION_AND_OR_PML     = .false.
NT_DUMP_ATTENUATION             = 500

# Instead of reconstructing the forward wavefield, this option reads it from the disk using asynchronous I/O.
# Outperforms conventional mode using a value of NTSTEP_BETWEEN_COMPUTE_KERNELS high enough.
NO_BACKWARD_RECONSTRUCTION      = .false.
```

## Sources

```toml
# source parameters
NSOURCES                        = 1              # number of sources (source information is then read from the DATA/SOURCE file)
force_normal_to_surface         = .false.        # angleforce normal to surface (external mesh and curve file needed)

# use an existing initial wave field as source or start from zero (medium initially at rest)
initialfield                    = .false.
add_Bielak_conditions_bottom    = .false.        # add Bielak conditions or not if initial plane wave
add_Bielak_conditions_right     = .false.
add_Bielak_conditions_top       = .false.
add_Bielak_conditions_left      = .false.

# acoustic forcing
ACOUSTIC_FORCING                = .false.        # acoustic forcing of an acoustic medium with a rigid interface

# noise simulations - type of noise source time function:
# 0=external (S_squared), 1=Ricker(second derivative), 2=Ricker(first derivative), 3=Gaussian, 4=Figure 2a of Tromp et al. 2010
# (default value 4 is chosen to reproduce the time function from Fig 2a of "Tromp et al., 2010, Noise Cross-Correlation Sensitivity Kernels")
noise_source_time_function_type = 4

# moving sources
# Set write_moving_sources_database to .true. if the generation of moving source databases takes
# a long time. Then the simulation is done in two steps: first you run the code and it writes the databases to file
# (in DATA folder by default). Then you rerun the code and it will read the databases in there directly possibly
# saving a lot of time.
# This is only useful for GPU version (for now)
write_moving_sources_database   = .false.
```

## Stations
- Output type of records can be displacement, velocity, pressure, etc.
 - You can change the interval seismograms are written to control I/O traffic.
 - `T0` controls the start time of the seismograms. By default, it is computed using half duration of the source.
 - You can export seismograms as `ascii`, `binary` (single of double precision), and in `SU` format.
 - You can define station location in the `Par_file` as sets or in the `DATA/STATIONS` file (when `use_exisiting_STATIONS = .true.`) 

```toml
# receiver set parameters for recording stations (i.e. recording points)
# seismotype : record 1=displ 2=veloc 3=accel 4=pressure 5=curl of displ 6=the fluid potential
seismotype                      = 1              # several values can be chosen. For example : 1,2,4

# interval in time steps for writing of seismograms
# every how many time steps we save the seismograms
# (costly, do not use a very small value; if you use a very large value that is larger than the total number
#  of time steps of the run, the seismograms will automatically be saved once at the end of the run anyway)
NTSTEP_BETWEEN_OUTPUT_SEISMOS   = 10000

# set to n to reduce the sampling rate of output seismograms by a factor of n
# defaults to 1, which means no down-sampling
NTSTEP_BETWEEN_OUTPUT_SAMPLE    = 1

# so far, this option can only be used if all the receivers are in acoustic elements
USE_TRICK_FOR_BETTER_PRESSURE   = .false.

# use this t0 as earliest starting time rather than the automatically calculated one
USER_T0                         = 0.0d0

# seismogram formats
save_ASCII_seismograms          = .true.         # save seismograms in ASCII format or not
save_binary_seismograms_single  = .true.         # save seismograms in single precision binary format or not (can be used jointly with ASCII above to save both)
save_binary_seismograms_double  = .false.        # save seismograms in double precision binary format or not (can be used jointly with both flags above to save all)
SU_FORMAT                       = .false.        # output single precision binary seismograms in Seismic Unix format (adjoint traces will be read in the same format)

# use an existing STATION file found in ./DATA or create a new one from the receiver positions below in this Par_file
use_existing_STATIONS           = .false.

# number of receiver sets (i.e. number of receiver lines to create below)
nreceiversets                   = 1

# orientation
anglerec                        = 0.d0           # angle to rotate components at receivers
rec_normal_to_surface           = .false.        # base anglerec normal to surface (external mesh and curve file needed)

# first receiver set (repeat these 6 lines and adjust nreceiversets accordingly)
nrec                            = 11             # number of receivers
xdeb                            = 300.           # first receiver x in meters
zdeb                            = 2200.          # first receiver z in meters
xfin                            = 3700.          # last receiver x in meters (ignored if only one receiver)
zfin                            = 2200.          # last receiver z in meters (ignored if only one receiver)
record_at_surface_same_vertical = .true.         # receivers inside the medium or at the surface (z values are ignored if this is set to true, they are replaced with the topography height)
```

## Adjoint Kernels

```toml
# save sensitivity kernels in ASCII format (much bigger files, but compatible with current GMT scripts) or in binary format
save_ASCII_kernels              = .true.

# since the accuracy of kernel integration may not need to respect the CFL, this option permits to save computing time, and memory with UNDO_ATTENUATION_AND_OR_PML mode
NTSTEP_BETWEEN_COMPUTE_KERNELS  = 1

# outputs approximate Hessian for preconditioning
APPROXIMATE_HESS_KL             = .false.
```

## Boundary Conditions

 - You can choose between `PML` and `Stacey` boundary conditions. Note that some simulation types do not support `PML` boundaries.
 - You can also add periodic boundaries.

```toml
# Perfectly Matched Layer (PML) boundaries
# absorbing boundary active or not
PML_BOUNDARY_CONDITIONS         = .true.
NELEM_PML_THICKNESS             = 3
ROTATE_PML_ACTIVATE             = .false.
ROTATE_PML_ANGLE                = 30.
# change the four parameters below only if you know what you are doing; they change the damping profiles inside the PMLs
K_MIN_PML                       = 1.0d0          # from Gedney page 8.11
K_MAX_PML                       = 1.0d0
damping_change_factor_acoustic  = 0.5d0
damping_change_factor_elastic   = 1.0d0
# set the parameter below to .false. unless you know what you are doing; this implements automatic adjustment of the PML parameters for elongated models.
 # The goal is to improve the absorbing efficiency of PML for waves with large incidence angles, but this can lead to artefacts.
# In particular, this option is efficient only when the number of sources NSOURCES is equal to one.
PML_PARAMETER_ADJUSTMENT        = .false.

# Stacey ABC
STACEY_ABSORBING_CONDITIONS     = .false.

# periodic boundaries
ADD_PERIODIC_CONDITIONS         = .false.
PERIODIC_HORIZ_DIST             = 4000.d0
```

## Model Material definitions
 - Model materials defined here can be used when the `MODEL=default`.

```toml
# number of model materials
nbmodels                        = 4
# available material types (see user manual for more information)
#   acoustic:              model_number 1 rho Vp 0  0 0 QKappa 9999 0 0 0 0 0 0 (for QKappa use 9999 to ignore it)
#   elastic:               model_number 1 rho Vp Vs 0 0 QKappa Qmu  0 0 0 0 0 0 (for QKappa and Qmu use 9999 to ignore them)
#   anisotropic:           model_number 2 rho c11 c13 c15 c33 c35 c55 c12 c23 c25   0 QKappa Qmu
#   anisotropic in AXISYM: model_number 2 rho c11 c13 c15 c33 c35 c55 c12 c23 c25 c22 QKappa Qmu
#   poroelastic:           model_number 3 rhos rhof phi c kxx kxz kzz Ks Kf Kfr etaf mufr Qmu
#   tomo:                  model_number -1 0 0 A 0 0 0 0 0 0 0 0 0 0
#
# note: When viscoelasticity or viscoacousticity is turned on,
#       the Vp and Vs values that are read here are the UNRELAXED ones i.e. the values at infinite frequency
#       unless the READ_VELOCITIES_AT_f0 parameter above is set to true, in which case they are the values at frequency f0.
#
#       Please also note that Qmu is always equal to Qs, but Qkappa is in general not equal to Qp.
#       To convert one to the other see doc/Qkappa_Qmu_versus_Qp_Qs_relationship_in_2D_plane_strain.pdf and
#       utils/attenuation/conversion_from_Qkappa_Qmu_to_Qp_Qs_from_Dahlen_Tromp_959_960.f90.
1 1 2700.d0 3000.d0 1732.051d0 0 0 9999 9999 0 0 0 0 0 0
2 1 2500.d0 2700.d0 0 0 0 9999 9999 0 0 0 0 0 0
3 1 2200.d0 2500.d0 1443.375d0 0 0 9999 9999 0 0 0 0 0 0
4 1 2200.d0 2200.d0 1343.375d0 0 0 9999 9999 0 0 0 0 0 0
```

## External Mesher Settings
```toml
# external tomography file
TOMOGRAPHY_FILE                 = ./DATA/tomo_file.xyz

# use an external mesh created by an external meshing tool or use the internal mesher
read_external_mesh              = .false.

#-----------------------------------------------------------
#
# PARAMETERS FOR EXTERNAL MESHING
#
#-----------------------------------------------------------

# data concerning mesh, when generated using third-party app (more info in README)
# (see also absorbing_conditions above)
mesh_file                       = ./DATA/mesh_file          # file containing the mesh
nodes_coords_file               = ./DATA/nodes_coords_file  # file containing the nodes coordinates
materials_file                  = ./DATA/materials_file     # file containing the material number for each element
free_surface_file               = ./DATA/free_surface_file  # file containing the free surface
axial_elements_file             = ./DATA/axial_elements_file   # file containing the axial elements if AXISYM is true
absorbing_surface_file          = ./DATA/absorbing_surface_file   # file containing the absorbing surface
acoustic_forcing_surface_file   = ./DATA/MSH/Surf_acforcing_Bottom_enforcing_mesh   # file containing the acoustic forcing surface
absorbing_cpml_file             = ./DATA/absorbing_cpml_file   # file containing the CPML element numbers
tangential_detection_curve_file = ./DATA/courbe_eros_nodes  # file containing the curve delimiting the velocity model
```

## Internal Mesher Parameters

-  Used when the `MODEL=default`.
- `xmin`, `xmax` defines the coordinates in X.
- `nx` defines the number of elements along X.
- Absorbing boundary conditions can be set in each side. If it is `.false.`, boundary acts as free surface.
- `interfacesfile` is a file defines the interfaces in Z direction.
- Regions are defined by their elements and the last value is the `material_number` refers back to Model Material section.

```toml
# file containing interfaces for internal mesh
interfacesfile                  = interfaces.dat

# geometry of the model (origin lower-left corner = 0,0) and mesh description
xmin                            = 0.d0           # abscissa of left side of the model
xmax                            = 4000.d0        # abscissa of right side of the model
nx                              = 80             # number of elements along X

# absorbing boundary parameters (see absorbing_conditions above)
absorbbottom                    = .true.
absorbright                     = .true.
absorbtop                       = .false.
absorbleft                      = .true.

# define the different regions of the model in the (nx,nz) spectral-element mesh
nbregions                       = 1              # then set below the different regions and model number for each region
# format of each line: nxmin nxmax nzmin nzmax material_number
1 80  1 60 1
```

Format of the `interfacesfile`:
```toml
number_of_interfaces

# interface number 1 (bottom of the mesh)
number_of_points_in_the_interface
x0 z0
x1 z1
...

# interface number 2
...
# interface number N (top of the mesh)
...

# Number of elements per layer
n_elements1
n_elements2
...
n_elementsN
```

Each model should have at least 2 interfaces (bottom and the top of the model).

Example simplest case where the bottom and top interfaces are flat. Thus contain only two points).

```toml
# number of interfaces
 2
#
# for each interface below, we give the number of points and then x,z for each point
#
# interface number 1 (bottom of the mesh)
 2
 0 0
 5000 0
# interface number 2 (topography, top of the mesh)
 2
    0 3000
 5000 3000
#
# for each layer, we give the number of spectral elements in the vertical direction
#
# layer number 1 (bottom layer)
 60
```

Example with curved topography and curved 2 additional inner interfaces:
```toml
# number of interfaces
 4
#
# for each interface below, we give the number of points and then x,z for each point
#
# interface number 1 (bottom of the mesh)
 2
 0 0
 5000 0
# interface number 2
 7
    0 1000
 1500 1100
 2000 1180
 2500 1200
 3000 1220
 3500 1170
 5000 1100
# interface number 3
 9
    0 2000
  500 2000
 1000 1900
 1500 1847
 2000 1900
 2500 2000
 3000 2090
 3500 2020
 5000 2000
# interface number 4 (topography, top of the mesh)
 8
    0 3000
  500 3000
 1000 3100
 1500 3350
 2000 3250
 2500 3180
 3000 3090
 5000 3000
#
# for each layer, we give the number of spectral elements in the vertical direction
#
# layer number 1 (bottom layer)
 20
# layer number 2
 20
# layer number 3 (top layer)
 20
```

## Display Parameters

- `NTSTEP_BETWEEN_OUTPUT_INFO` controls interval which the info is printed by the solver.
- Grid can be outputted for gnuplot or in  ascii format.
- Color jpg outputs of the wavefield snapshots can be used (in displacement, velocity, accelaration format).
- Vector field snapshots can be exported in `Postscript` format.
- Also, wavefields can be dumped as binary files.

```toml
# interval at which we output time step info and max of norm of displacement
# (every how many time steps we display information about the simulation. costly, do not use a very small value)
NTSTEP_BETWEEN_OUTPUT_INFO      = 100

# meshing output
output_grid_Gnuplot             = .false.        # generate a GNUPLOT file containing the grid, and a script to plot it
output_grid_ASCII               = .false.        # dump the grid in an ASCII text file consisting of a set of X,Y,Z points or not

# to plot total energy curves, for instance to monitor how CPML absorbing layers behave;
# should be turned OFF in most cases because a bit expensive
OUTPUT_ENERGY                   = .false.

# every how many time steps we compute energy (which is a bit expensive to compute)
NTSTEP_BETWEEN_OUTPUT_ENERGY    = 10

# Compute the field int_0^t v^2 dt for a set of GLL points and write it to file. Use
# the script utils/visualisation/plotIntegratedEnergyFile.py to watch. It is refreshed at the same time than the seismograms
COMPUTE_INTEGRATED_ENERGY_FIELD = .false.

#-----------------------------------------------------------
#
# Movies/images/snaphots visualizations
#
#-----------------------------------------------------------

# every how many time steps we draw JPEG or PostScript pictures of the simulation
# and/or we dump results of the simulation as ASCII or binary files (costly, do not use a very small value)
NTSTEP_BETWEEN_OUTPUT_IMAGES    = 100

# minimum amplitude kept in % for the JPEG and PostScript snapshots; amplitudes below that are muted
cutsnaps                        = 1.

#### for JPEG color images ####
output_color_image              = .true.         # output JPEG color image of the results every NTSTEP_BETWEEN_OUTPUT_IMAGES time steps or not
imagetype_JPEG                  = 2              # display 1=displ_Ux 2=displ_Uz 3=displ_norm 4=veloc_Vx 5=veloc_Vz 6=veloc_norm 7=accel_Ax 8=accel_Az 9=accel_norm 10=pressure
factor_subsample_image          = 1.0d0          # (double precision) factor to subsample or oversample (if set to e.g. 0.5) color images output by the code (useful for very large models, or to get nicer looking denser pictures)
USE_CONSTANT_MAX_AMPLITUDE      = .false.        # by default the code normalizes each image independently to its maximum; use this option to use the global maximum below instead
CONSTANT_MAX_AMPLITUDE_TO_USE   = 1.17d4         # constant maximum amplitude to use for all color images if the above USE_CONSTANT_MAX_AMPLITUDE option is true
POWER_DISPLAY_COLOR             = 0.30d0         # non linear display to enhance small amplitudes in JPEG color images
DRAW_SOURCES_AND_RECEIVERS      = .true.         # display sources as orange crosses and receivers as green squares in JPEG images or not
DRAW_WATER_IN_BLUE              = .true.         # display acoustic layers as constant blue in JPEG images, because they likely correspond to water in the case of ocean acoustics or in the case of offshore oil industry experiments (if off, display them as greyscale, as for elastic or poroelastic elements, for instance for acoustic-only oil industry models of solid media)
USE_SNAPSHOT_NUMBER_IN_FILENAME = .false.        # use snapshot number in the file name of JPEG color snapshots instead of the time step (for instance to create movies in an easier way later)

#### for PostScript snapshots ####
output_postscript_snapshot      = .false.         # output Postscript snapshot of the results every NTSTEP_BETWEEN_OUTPUT_IMAGES time steps or not
imagetype_postscript            = 1              # display 1=displ vector 2=veloc vector 3=accel vector; small arrows are displayed for the vectors
meshvect                        = .true.         # display mesh on PostScript plots or not
modelvect                       = .false.        # display velocity model on PostScript plots or not
boundvect                       = .true.         # display boundary conditions on PostScript plots or not
interpol                        = .true.         # interpolation of the PostScript display on a regular grid inside each spectral element, or use the non-evenly spaced GLL points
pointsdisp                      = 6              # number of points in each direction for interpolation of PostScript snapshots (set to 1 for lower-left corner only)
subsamp_postscript              = 1              # subsampling of background velocity model in PostScript snapshots
sizemax_arrows                  = 1.d0           # maximum size of arrows on PostScript plots in centimeters
US_LETTER                       = .false.        # use US letter or European A4 paper for PostScript plots

#### for wavefield dumps ####
output_wavefield_dumps          = .false.        # output wave field to a text file (creates very big files)
imagetype_wavefield_dumps       = 1              # display 1=displ vector 2=veloc vector 3=accel vector 4=pressure
use_binary_for_wavefield_dumps  = .false.        # use ASCII or single-precision binary format for the wave field dumps
```

## Other Options

 - Multiple simulations can be run together using `NUMBER_OF_SIMULTANEOUS_RUNS`  set to some number `N`. These run require additional directory structures (`run0001`, `run0002`, ..., `run000N`). If the mesh and model is the same between the simulations and only change is the `STATIONS` and sources `BROADCAST_SAME_MESH_AND_MODEL` can be set to `.true.`.
 - `GPU_MODE` can be set to `.true.` if it is available. For this support code needs to be compiled with `cuda support` (`./configure --with-cuda`).

```toml
NUMBER_OF_SIMULTANEOUS_RUNS     = 1

BROADCAST_SAME_MESH_AND_MODEL   = .true.

GPU_MODE                        = .false.
```

# SOURCE File (DATA/SOURCE)

```toml
## Source 1
source_surf                     = .false.        # source inside the medium, or source automatically moved exactly at the surface by the solver
xs                              = 2500.          # source location x in meters
zs                              = 2500.          # source location z in meters (zs is ignored if source_surf is set to true, it is replaced with the topography height)
## Source type parameters:
#  1 = elastic force or acoustic pressure
#  2 = moment tensor
# or Initial field type (when initialfield set in Par_file):
# For a plane wave including converted and reflected waves at the free surface:
#  1 = P wave,
#  2 = S wave,
#  3 = Rayleigh wave
# For a plane wave without converted nor reflected waves at the free surface, i.e. with the incident wave only:
#  4 = P wave,
#  5 = S wave
# For initial mode displacement:
#  6 = mode (2,3) of a rectangular membrane
source_type                     = 1
# Source time function:
# In the case of a source located in an acoustic medium,
# to get pressure for a Ricker in the seismograms, here we need to select a Gaussian for the potential Chi
# used as a source, rather than a Ricker, because pressure = - Chi_dot_dot.
# This is true both when USE_TRICK_FOR_BETTER_PRESSURE is set to .true. or to .false.
# Options:
#  1 = second derivative of a Gaussian (a.k.a. Ricker),
#  2 = first derivative of a Gaussian,
#  3 = Gaussian,
#  4 = Dirac,
#  5 = Heaviside (4 and 5 will produce noisy recordings because of frequencies above the mesh resolution limit),
#  6 = ocean acoustics type I,
#  7 = ocean acoustics type II,
#  8 = external source time function = 8 (source read from file),
#  9 = burst,
# 10 = Sinus source time function,
# 11 = Marmousi Ormsby wavelet
time_function_type              = 1
# If time_function_type == 8, enter below the custom source file to read (two columns file with time and amplitude) :
# (For the moment dt must be equal to the dt of the simulation. File name cannot exceed 150 characters)
# IMPORTANT: do NOT put quote signs around the file name, just put the file name itself otherwise the run will stop
name_of_source_file             = YYYYYYYY       # Only for option 8 : file containing the source wavelet
burst_band_width                = 0.             # Only for option 9 : band width of the burst
f0                              = 10.0           # dominant source frequency (Hz) if not Dirac or Heaviside
tshift                          = 0.0            # time shift when multi sources (if one source, must be zero)
## Force source
# angle of the source (for a force only); for a plane wave, this is the incidence angle; for moment tensor sources this is unused
anglesource                     = 0.
## Moment tensor
# The components of a moment tensor source must be given in N.m, not in dyne.cm as in the DATA/CMTSOLUTION source file of the 3D version of the code.
Mxx                             = 1.             # Mxx component (for a moment tensor source only)
Mzz                             = 1.             # Mzz component (for a moment tensor source only)
Mxz                             = 0.             # Mxz component (for a moment tensor source only)
## Amplification (factor to amplify source time function)
factor                          = 1.d10          # amplification factor
## Moving source parameters
vx                              = 0.0            # Horizontal source velocity (m/s)
vz                              = 0.0            # Vertical source velocity (m/s)
```

# STATIONS File (DATA/STATIONS)

 - if `use_existing_STATIONS=.true.`, it is required. Else, it is created automatically.
 - Format is: `network station x z depth elevation`. Depth and elevation values are not used.
 - Same format is used in `3D` code. Then `x` is `latitude`, `z` is longitude. 

```
S0001    AA          300.0000000         3000.0000000       0.0         0.0
S0002    AA          640.0000000         3000.0000000       0.0         0.0
S0003    AA          980.0000000         3000.0000000       0.0         0.0
S0004    AA         1320.0000000         3000.0000000       0.0         0.0
S0005    AA         1660.0000000         3000.0000000       0.0         0.0
S0006    AA         2000.0000000         3000.0000000       0.0         0.0
S0007    AA         2340.0000000         3000.0000000       0.0         0.0
S0008    AA         2680.0000000         3000.0000000       0.0         0.0
S0009    AA         3020.0000000         3000.0000000       0.0         0.0
S0010    AA         3360.0000000         3000.0000000       0.0         0.0
S0011    AA         3700.0000000         3000.0000000       0.0         0.0
```